#  Siniestros Viales en Bogotá D.C. (2015–2021)

## Descripción del Proyecto

Análisis exploratorio e interactivo de los **siniestros viales georreferenciados** ocurridos en Bogotá desde 2015, con datos de la **Secretaría Distrital de Movilidad**.

Este proyecto busca responder preguntas concretas sobre seguridad vial:
- ¿En qué localidades ocurren más accidentes?
- ¿En qué horas y días son más frecuentes?
- ¿Cómo ha evolucionado la accidentalidad año a año?
- ¿Dónde se concentran los siniestros más graves?

---

##  Fuente de Datos

**Datos Abiertos Bogotá — Secretaría Distrital de Movilidad**  
Licencia: Creative Commons Attribution 4.0  
Enlace: https://datosabiertos.bogota.gov.co/dataset/historico-siniestros-bogota-d-c

---

##  Visualizaciones

| # | Gráfico | Tipo |
|---|---------|------|
| 1 | Siniestros por localidad | Bar Chart horizontal |
| 2 | Distribución por gravedad | Pie Chart |
| 3 | Tendencia histórica anual | Line Chart |
| 4 | Calor por hora y día de semana | Heatmap |
| 5 | Clase de accidente por localidad | Stacked Bar |
| 6 | Mapa de puntos críticos | Scatter Mapbox |


In [87]:
# Instalación (solo si no tienes las librerías)
# !pip install plotly pandas

In [88]:
# Importar librerías
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

print('Librerías cargadas ')

Librerías cargadas 


## 1. Carga de Datos

In [89]:
# URL directa del dataset (Datos Abiertos Bogotá)
URL = 'https://datosabiertos.bogota.gov.co/dataset/8624f916-1db2-4c17-b669-19a19b35d1ca/resource/f5862aaa-4e1c-463e-94d5-f04db8164360/download/historico_siniestros_bogota_d.c_-.csv'

df = pd.read_csv('historico_siniestros_bogota_d.c_-.csv', encoding='latin-1')

print(f'\nRegistros: {len(df):,}')
print(f'Columnas:  {df.shape[1]}')
print(f'\nColumnas disponibles:')
for col in df.columns:
    print(f'  - {col}')


Registros: 199,146
Columnas:  16

Columnas disponibles:
  - ï»¿X
  - Y
  - OBJECTID
  - FORMULARIO
  - CODIGO_ACCIDENTE
  - FECHA_OCURRENCIA_ACC
  - ANO_OCURRENCIA_ACC
  - DIRECCION
  - GRAVEDAD
  - CLASE_ACC
  - LOCALIDAD
  - FECHA_HORA_ACC
  - LATITUD
  - LONGITUD
  - CIV
  - PK_CALZADA


In [90]:
# Vista previa de los datos
df.head()

,ï»¿X,Y,OBJECTID,FORMULARIO,CODIGO_ACCIDENTE,FECHA_OCURRENCIA_ACC,ANO_OCURRENCIA_ACC,DIRECCION,GRAVEDAD,CLASE_ACC,LOCALIDAD,FECHA_HORA_ACC,LATITUD,LONGITUD,CIV,PK_CALZADA
0,-74.090924,4.693807,1,A000640275,4484660,2017/06/12 00:00:00+00,2017,AV AVENIDA BOYACA-CL 79 02,SOLO DANOS,CHOQUE,ENGATIVA,2017/06/12 05:30:00+00,4.693807,-74.090924,10006772.0,221236.0
1,-74.121000,4.603000,2,A001233353,10533499,2020/11/19 00:00:00+00,2020,CL 26 S- KR 50 02,CON HERIDOS,OTRO,PUENTE ARANDA,2020/11/19 02:05:00+00,4.603000,-74.121000,16004560.0,NaN
2,-74.042000,4.682000,4,A001232786,10533629,2020/11/10 00:00:00+00,2020,KR 9 - CL 100 02,SOLO DANOS,CHOQUE,USAQUEN,2020/11/10 13:30:00+00,4.682000,-74.042000,30001107.0,NaN
3,-74.166937,4.587187,7,A000200705,4412699,2015/05/11 00:00:00+00,2015,CL 63A-KR 72 S 02,SOLO DANOS,CHOQUE,CIUDAD BOLIVAR,2015/05/11 10:50:00+00,4.587187,-74.166937,19001483.0,136166.0
4,-74.092901,4.607648,8,A000402862,4447845,2016/06/08 00:00:00+00,2016,KR 27-CL 9 14,SOLO DANOS,CHOQUE,LOS MARTIRES,2016/06/08 21:30:00+00,4.607648,-74.092901,14000548.0,239719.0


## 2. Limpieza y Preparación

In [91]:
# Verificar valores nulos
print('Valores nulos por columna:')
print(df.isnull().sum()[df.isnull().sum() > 0])
print(f'\nTotal registros: {len(df):,}')

Valores nulos por columna:
LOCALIDAD        46
CIV            1701
PK_CALZADA    37974
dtype: int64

Total registros: 199,146


In [92]:
# Estandarizar nombres de columnas a mayúsculas para mayor robustez
df.columns = df.columns.str.upper().str.strip()

# Identificar columnas clave automáticamente (elif evita sobreescritura)
col_map = {}
for col in df.columns:
    if 'FECHA' in col and 'fecha' not in col_map:       col_map['fecha']     = col
    elif 'HORA' in col and 'hora' not in col_map:       col_map['hora']      = col
    if 'LOCAL' in col and 'DEPAR' not in col and 'localidad' not in col_map:
                                                         col_map['localidad'] = col
    if 'GRAVE' in col and 'gravedad' not in col_map:    col_map['gravedad']  = col
    if 'CLASE' in col and 'clase' not in col_map:       col_map['clase']     = col
    if 'LAT' in col and 'lat' not in col_map:           col_map['lat']       = col
    if ('LON' in col or 'LNG' in col) and 'lon' not in col_map:
                                                         col_map['lon']       = col
    # Fallback por nombre exacto
    col_map.setdefault('lon', col) if col in ['X', 'LONGITUD'] else None
    col_map.setdefault('lat', col) if col in ['Y', 'LATITUD']  else None

print('Columnas identificadas:')
for k, v in col_map.items():
    print(f'  {k:12} → {v}')

columnas_faltantes = [k for k in ['fecha','localidad','gravedad'] if k not in col_map]
if columnas_faltantes:
    print(f'\n  Columnas clave no detectadas: {columnas_faltantes}')
    print('   Revisa los nombres en el dataset y ajusta col_map manualmente si es necesario.')
else:
    print('\n Todas las columnas clave detectadas.')


Columnas identificadas:
  lat          → Y
  fecha        → FECHA_OCURRENCIA_ACC
  gravedad     → GRAVEDAD
  clase        → CLASE_ACC
  localidad    → LOCALIDAD
  hora         → FECHA_HORA_ACC
  lon          → LONGITUD

 Todas las columnas clave detectadas.


In [93]:
# Convertir fecha y extraer componentes temporales
if 'fecha' in col_map:
    df[col_map['fecha']] = pd.to_datetime(df[col_map['fecha']], errors='coerce')
    df['AÑO']     = df[col_map['fecha']].dt.year
    df['MES']     = df[col_map['fecha']].dt.month
    df['DIA_SEM'] = df[col_map['fecha']].dt.day_name()
    print('Fecha procesada ')
else:
    print('  Columna de fecha no encontrada en col_map.')

# Extraer hora — soporta formatos HH:MM y HH:MM:SS
if 'hora' in col_map:
    hora_str = df[col_map['hora']].astype(str).str.strip()
    df['HORA_NUM'] = pd.to_datetime(hora_str, format='%H:%M:%S', errors='coerce').dt.hour
    # Fallback para formato HH:MM
    mask_null = df['HORA_NUM'].isna()
    df.loc[mask_null, 'HORA_NUM'] = pd.to_datetime(
        hora_str[mask_null], format='%H:%M', errors='coerce'
    ).dt.hour
    cobertura = df['HORA_NUM'].notna().sum()
    print(f'Hora procesada  — {cobertura:,} registros con hora válida')

# Filtrar rango de años válido
if 'AÑO' in df.columns:
    anio_min, anio_max = int(df['AÑO'].min()), int(df['AÑO'].max())
    df = df[df['AÑO'].between(2015, 2025)].copy()
    print(f'\nRegistros tras limpieza: {len(df):,}')
    print(f'Rango de años: {df["AÑO"].min():.0f} – {df["AÑO"].max():.0f}')


Fecha procesada 
Hora procesada  — 0 registros con hora válida

Registros tras limpieza: 199,146
Rango de años: 2015 – 2021


In [94]:
# Crear carpeta para exportar gráficos
os.makedirs('graficos_interactivos', exist_ok=True)
print(" Carpeta 'graficos_interactivos' lista.")

 Carpeta 'graficos_interactivos' lista.


---

## 3. Análisis y Visualizaciones

### 3.1  ¿En qué localidades ocurren más siniestros?

In [95]:
col_loc = col_map.get('localidad', 'LOCALIDAD')

# Compatible con pandas 1.x y 2.x
top_localidades = (
    df[col_loc]
    .value_counts()
    .head(20)
    .reset_index()
)
top_localidades.columns = ['Localidad', 'Siniestros']
top_localidades = top_localidades.sort_values('Siniestros')

fig1 = px.bar(
    top_localidades,
    x='Siniestros',
    y='Localidad',
    orientation='h',
    color='Siniestros',
    color_continuous_scale=[
        [0.0, '#fde68a'],
        [0.5, '#f97316'],
        [1.0, '#991b1b']
    ],
    text='Siniestros',
    title=' Top 20 Localidades con Más Siniestros Viales — Bogotá',
    labels={'Siniestros': 'Total de siniestros'}
)

fig1.update_traces(
    texttemplate='%{text:,}',
    textposition='outside'
)

fig1.update_layout(
    height=580,
    template='plotly_white',
    title_font_size=17,
    coloraxis_showscale=False,
    xaxis=dict(showgrid=True, gridcolor='#f0f0f0'),
    yaxis_title='',
    font_family='Arial'
)

fig1.show()
fig1.write_html('graficos_interactivos/01_siniestros_por_localidad.html')
print(' Guardado: 01_siniestros_por_localidad.html')

 Guardado: 01_siniestros_por_localidad.html


Las localidades con mayor accidentalidad son las que concentran el mayor flujo vehicular y densidad poblacional de Bogotá. **Suba, Kennedy y Engativá** aparecen consistentemente entre las primeras, junto con **Rafael Uribe Uribe, Tunjuelito y Usme** en el sur de la ciudad. **Usaquén** también figura entre las más afectadas, explicado por su alta actividad comercial y el intenso tráfico de la zona norte. Llama la atención la presencia de **Teusaquillo y Chapinero**, localidades de tamaño medio pero con alta circulación de vehículos por concentrar zonas comerciales y universitarias. **Sumapaz** registra los valores más bajos, coherente con su carácter rural y baja densidad de tráfico.

---

### 3.2  ¿Cómo se distribuyen los siniestros por gravedad?

In [96]:
col_grav = col_map.get('gravedad', 'GRAVEDAD')

# Compatible con pandas 1.x y 2.x
gravedad_counts = df[col_grav].value_counts().reset_index()
gravedad_counts.columns = ['Gravedad', 'Total']

color_grav = {
    'SOLO DANOS':   '#3b82f6',
    'CON HERIDOS':  '#f59e0b',
    'CON MUERTOS':  '#18181b',
    'Solo Daños':   '#3b82f6',
    'Herido':       '#f59e0b',
    'Muerto':       '#18181b',
}

fig2 = go.Figure(go.Pie(
    labels=gravedad_counts['Gravedad'],
    values=gravedad_counts['Total'],
    hole=0.45,
    marker_colors=[
        color_grav.get(str(g).strip(), '#94a3b8')
        for g in gravedad_counts['Gravedad']
    ],
    textinfo='percent+label',
    textfont_size=13,
    hovertemplate='<b>%{label}</b><br>Total: %{value:,}<br>Porcentaje: %{percent}<extra></extra>'
))

fig2.add_annotation(
    text=f"<b>{len(df):,}</b><br>siniestros",
    x=0.5, y=0.5, showarrow=False,
    font=dict(size=14, color='#374151')
)

fig2.update_layout(
    title=' Distribución de Siniestros por Gravedad — Bogotá',
    title_font_size=17,
    height=450,
    template='plotly_white',
    font_family='Arial',
    legend=dict(orientation='h', y=-0.1, x=0.25)
)

fig2.show()
fig2.write_html('graficos_interactivos/02_distribucion_gravedad.html')
print(' Guardado: 02_distribucion_gravedad.html')

 Guardado: 02_distribucion_gravedad.html


La mayoría de los siniestros resultan en solo daños materiales. Sin embargo, los accidentes con heridos y muertos representan un porcentaje significativo que requiere atención prioritaria en políticas de seguridad vial.

---

### 3.3  ¿Cómo ha evolucionado la accidentalidad año a año?

In [97]:
col_grav = col_map.get('gravedad', 'GRAVEDAD')

tendencia = (
    df.groupby(['AÑO', col_grav])
    .size()
    .reset_index(name='Total')
)

tendencia_total = df.groupby('AÑO').size().reset_index(name='Total')

y_max = tendencia_total['Total'].max() * 1.15

# Rango fijo del eje X — igual en ambos subplots para alinear los años
anios_disponibles = sorted(df['AÑO'].dropna().unique().astype(int))
x_range = [anios_disponibles[0] - 0.3, anios_disponibles[-1] + 0.3]

fig3 = make_subplots(
    rows=2, cols=1,
    subplot_titles=(
        'Total anual de siniestros viales',
        'Evolución por gravedad'
    ),
    shared_xaxes=True,   # ← comparte el eje X entre los dos subplots
    vertical_spacing=0.12
)

# Gráfico superior — Total anual
fig3.add_trace(
    go.Scatter(
        x=tendencia_total['AÑO'],
        y=tendencia_total['Total'],
        mode='lines+markers+text',
        text=tendencia_total['Total'].apply(lambda x: f'{x:,}'),
        textposition='top center',
        line=dict(color='#1d4ed8', width=3),
        marker=dict(size=9),
        fill='tozeroy',
        fillcolor='rgba(29,78,216,0.08)',
        name='Total',
        hovertemplate='Año: %{x}<br>Total: %{y:,}<extra></extra>'
    ),
    row=1, col=1
)

# Gráfico inferior — Por gravedad
color_lines = {
    'Solo Daños': '#3b82f6', 'Herido': '#f59e0b', 'Muerto': '#18181b',
    'SOLO DANOS': '#3b82f6', 'CON HERIDOS': '#f59e0b', 'CON MUERTOS': '#18181b'
}

for grav in tendencia[col_grav].unique():
    sub = tendencia[tendencia[col_grav] == grav]
    fig3.add_trace(
        go.Scatter(
            x=sub['AÑO'], y=sub['Total'],
            mode='lines+markers',
            name=str(grav),
            line=dict(color=color_lines.get(str(grav).strip(), '#94a3b8'), width=2.5),
            marker=dict(size=7),
            hovertemplate=f'<b>{grav}</b><br>Año: %{{x}}<br>Total: %{{y:,}}<extra></extra>'
        ),
        row=2, col=1
    )

fig3.update_layout(
    height=640,
    template='plotly_white',
    title_text=' Evolución Histórica de Siniestros Viales — Bogotá',
    title_font_size=17,
    font_family='Arial',
    hovermode='x unified',
    legend=dict(orientation='h', y=-0.08)
)

# Aplicar el mismo rango y ticks a AMBOS ejes X
fig3.update_xaxes(
    showgrid=True, gridcolor='#f0f0f0',
    dtick=1,
    tickmode='array',
    tickvals=anios_disponibles,
    ticktext=[str(a) for a in anios_disponibles],
    range=x_range
)
fig3.update_yaxes(showgrid=True, gridcolor='#f0f0f0')
fig3.update_yaxes(range=[0, y_max], row=1, col=1)

fig3.show()
fig3.write_html('graficos_interactivos/03_tendencia_historica.html')
print(' Guardado: 03_tendencia_historica.html')

 Guardado: 03_tendencia_historica.html


La tendencia histórica muestra una caída notable en 2020, consistente con las restricciones de movilidad por la pandemia de COVID-19. Los años previos muestran un comportamiento relativamente estable con ligeras variaciones anuales.

---

### 3.4  ¿En qué horas y días ocurren más accidentes?

In [98]:
if 'hora' in col_map and 'DIA_SEM' in df.columns:

    # Extraer hora — formato '2017/06/12 05:30:00+00'
    df['HORA_NUM'] = (
        pd.to_datetime(
            df[col_map['hora']].astype(str).str.replace('+00', '+0000', regex=False),
            format='%Y/%m/%d %H:%M:%S%z', errors='coerce'
        ).dt.hour
    )

    validos = df['HORA_NUM'].notna().sum()
    print(f'Registros con hora válida: {validos:,}')

    if validos == 0:
        print(' No hay datos de hora — el heatmap no puede generarse.')
        print(f'   Valores únicos: {df[col_map["hora"]].dropna().unique()[:5]}')
    else:
        dias_orden = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
        dias_es    = ['Lunes','Martes','Miércoles','Jueves','Viernes','Sábado','Domingo']
        dia_map    = dict(zip(dias_orden, dias_es))

        df['DIA_ES'] = df['DIA_SEM'].map(dia_map).fillna(df['DIA_SEM'])

        heatmap_data = (
            df.dropna(subset=['HORA_NUM', 'DIA_ES'])
            .assign(HORA_NUM=lambda d: d['HORA_NUM'].astype(int))
            .groupby(['DIA_ES', 'HORA_NUM'])
            .size()
            .reset_index(name='Total')
            .pivot(index='DIA_ES', columns='HORA_NUM', values='Total')
            .reindex(columns=range(24), fill_value=0)
            .fillna(0)
            .reindex([d for d in dias_es if d in df['DIA_ES'].unique()])
        )

        fig4 = go.Figure(go.Heatmap(
            z=heatmap_data.values,
            x=[f'{h:02d}:00' for h in range(24)],
            y=heatmap_data.index.tolist(),
            colorscale=[
                [0.0, '#f0f9ff'],
                [0.3, '#fde68a'],
                [0.6, '#f97316'],
                [1.0, '#7f1d1d']
            ],
            hovertemplate='Día: %{y}<br>Hora: %{x}<br>Siniestros: %{z:,}<extra></extra>',
            colorbar=dict(title='Siniestros')
        ))

        fig4.update_layout(
            title=' Mapa de Calor — Siniestros por Día y Hora',
            title_font_size=17,
            height=400,
            template='plotly_white',
            font_family='Arial',
            xaxis_title='Hora del día',
            yaxis_title=''
        )

        fig4.show()
        fig4.write_html('graficos_interactivos/04_heatmap_hora_dia.html')
        print(' Guardado: 04_heatmap_hora_dia.html')

else:
    print(' Columnas de hora o día no disponibles.')

Registros con hora válida: 199,146


 Guardado: 04_heatmap_hora_dia.html


El mapa de calor revela que la accidentalidad se concentra principalmente en la franja horaria de 6:00 a.m. a 9:00 p.m., con los niveles más altos entre las 12:00 y las 15:00 horas. Los días entre semana (lunes a viernes) muestran una actividad más intensa en horas laborales, mientras que los sábados y domingos presentan un patrón más extendido hacia la noche. Las madrugadas (0:00–4:00) registran la menor accidentalidad en todos los días.

---

### 3.5  ¿Qué tipo de accidentes predominan en cada localidad?

In [99]:
col_loc   = col_map.get('localidad', 'LOCALIDAD')
col_clase = col_map.get('clase', 'CLASE_ACCIDENTE')

if col_clase in df.columns:

    top10_loc = df[col_loc].value_counts().head(10).index.tolist()

    clase_loc = (
        df[df[col_loc].isin(top10_loc)]
        .groupby([col_loc, col_clase])
        .size()
        .reset_index(name='Total')
    )

    fig5 = px.bar(
        clase_loc,
        x=col_loc,
        y='Total',
        color=col_clase,
        barmode='stack',
        color_discrete_sequence=px.colors.qualitative.Bold,
        title=' Tipo de Accidente por Localidad (Top 10)',
        labels={
            col_loc: '',
            'Total': 'Número de siniestros',
            col_clase: 'Tipo'
        }
    )

    fig5.update_layout(
        height=560,
        template='plotly_white',
        title_font_size=17,
        font_family='Arial',
        xaxis_tickangle=-35,
        margin=dict(b=160),
        legend=dict(
            orientation='h',
            y=-0.42,
            x=0.5,
            xanchor='center',
            title_text='Tipo'
        ),
        yaxis=dict(showgrid=True, gridcolor='#f0f0f0')
    )

    fig5.show()
    fig5.write_html('graficos_interactivos/05_clase_por_localidad.html')
    print(' Guardado: 05_clase_por_localidad.html')

else:
    print(f' Columna de clase de accidente no encontrada. Columnas disponibles: {df.columns.tolist()}')

 Guardado: 05_clase_por_localidad.html


El choque es con una gran diferencia el tipo de accidente más frecuente en todas las localidades, representando la gran mayoría de los siniestros. El atropello aparece como segundo tipo pero en proporciones mucho menores. Kennedy y Engativá lideran en volumen total, coherente con su alta densidad vehicular. Tipos como volcamiento, caída de ocupante e incendio tienen una presencia marginal en todas las localidades.

---

### 3.6  Mapa de Puntos Críticos de Accidentalidad

In [100]:
col_lat = col_map.get('lat')
col_lon = col_map.get('lon')

if col_lat and col_lon:

    col_grav = col_map.get('gravedad', 'GRAVEDAD')
    col_loc  = col_map.get('localidad', 'LOCALIDAD')

    # Solo registros con coordenadas válidas en Bogotá
    df_map = df[
        df[col_lat].between(4.45, 4.85) &
        df[col_lon].between(-74.30, -73.95)
    ].dropna(subset=[col_lat, col_lon]).copy()

    # Muestra representativa para no sobrecargar el mapa
    if len(df_map) > 20000:
        df_map = df_map.sample(20000, random_state=42)

        color_map_grav = {
            'Solo Daños':  '#22c55e',   # verde
            'SOLO DANOS':  '#22c55e',
            'Herido':      '#facc15',   # amarillo
            'CON HERIDOS': '#facc15',
            'Muerto':      '#ef4444',   # rojo
            'CON MUERTOS': '#ef4444',
}

    df_map['COLOR'] = df_map[col_grav].map(
        lambda x: color_map_grav.get(str(x).strip(), '#94a3b8')
    )

    fig6 = px.scatter_mapbox(
        df_map,
        lat=col_lat,
        lon=col_lon,
        color=col_grav,
        hover_name=col_loc if col_loc in df_map.columns else None,
        hover_data={col_grav: True, col_lat: False, col_lon: False},
        color_discrete_map={
            'Solo Daños':  '#22c55e',
            'SOLO DANOS':  '#22c55e',
            'Herido':      '#facc15',
            'CON HERIDOS': '#facc15',
            'Muerto':      '#ef4444',
            'CON MUERTOS': '#ef4444',
},
        opacity=0.5,
        zoom=10.5,
        center=dict(lat=4.65, lon=-74.08),
        title=' Mapa de Siniestros Viales — Bogotá D.C.',
        labels={col_grav: 'Gravedad'}
    )

    fig6.update_layout(
        mapbox_style='carto-positron',
        height=600,
        title_font_size=17,
        font_family='Arial',
        legend=dict(orientation='h', y=-0.05)
    )

    fig6.update_traces(marker=dict(size=4))

    fig6.show()
    fig6.write_html('graficos_interactivos/06_mapa_siniestros.html')
    print(' Guardado: 06_mapa_siniestros.html')

else:
    print('⚠️ Columnas de coordenadas no encontradas. Revisa los nombres en col_map.')

 Guardado: 06_mapa_siniestros.html


El mapa revela corredores viales de alta accidentalidad, especialmente sobre las avenidas principales como la Av. NQS, Av. El Dorado y la Av. 1ro de Mayo. Los puntos rojos (muertos) se concentran en intersecciones de alta velocidad y vías de acceso rápido.

> 🔍 Haz zoom en el mapa para explorar puntos críticos específicos por barrio.

---

## 4. Conclusiones

El análisis de los siniestros viales en Bogotá revela patrones claros y accionables:

**1. Concentración geográfica**  
Kennedy y Engativá lideran en número de siniestros, seguidas de Suba, Usaquén y Fontibón. Esta concentración responde directamente a su alta densidad vehicular y poblacional. Localidades del sur como Bosa y Tunjuelito también figuran entre las más afectadas, lo que evidencia que la problemática no es exclusiva de las zonas de mayor ingreso.

**2. El choque domina sobre todos los demás tipos**  
En todas las localidades, el choque representa la gran mayoría de los siniestros. El atropello aparece como segundo tipo pero a gran distancia. Volcamientos, caídas de ocupante e incendios tienen presencia marginal. Esto apunta a que las intervenciones de mayor impacto deben enfocarse en reducir colisiones entre vehículos.

**3. La franja horaria de mayor riesgo es más amplia de lo esperado**  
La accidentalidad no se limita a las horas pico tradicionales: se mantiene alta desde las **6:00 a.m. hasta las 9:00 p.m.**, con el nivel más elevado entre las **12:00 y las 15:00 horas**. Los fines de semana el patrón se extiende hacia la noche, sugiriendo factores adicionales como el ocio y el consumo de alcohol.

**4. El COVID-19 transformó la movilidad**  
La caída en accidentalidad en 2020 es el hecho más marcado de la serie histórica — evidencia directa del impacto de las restricciones de circulación. La tendencia descendente continúa en 2021, lo que puede reflejar tanto cambios en los patrones de movilidad como la adopción del teletrabajo.

**5. Los corredores viales principales concentran los siniestros más graves**  
El mapa evidencia que los puntos rojos (muertos) y amarillos (heridos) siguen las trazas de las avenidas principales de Bogotá. Las intervenciones de infraestructura en estos corredores — señalización, velocidades controladas, separadores — tendrían el mayor impacto en reducir la severidad de los accidentes.

---
*Datos: Secretaría Distrital de Movilidad — Datos Abiertos Bogotá*  
*Licencia: Creative Commons Attribution 4.0*  
*Análisis realizado con Python, Plotly Express y pandas.*
